# Week 1: Foundations of Conventional Neural Netweorks

Implement the foundational layers of CNNs (pooling, convolutions) and stack them properly in a deep network to solve multi-class image classification problems.

**Learning Objectives**:

* Explain the convolution operation
* Apply two different types of pooling operations
* Identify the components used in a convolutional neural network (padding, stride, filter, ...) and their purpose
* Build a convolutional neural network
* Implement convolutional and pooling layers in numpy, including forward propagation
* Implement helper functions to use when implementing a TensorFlow model
* Create a mood classifer using the TF Keras Sequential API
* Build a ConvNet to identify sign language digits using the TF Keras Functional API
* Build and train a ConvNet in TensorFlow for a binary classification problem
* Build and train a ConvNet in TensorFlow for a multiclass classification problem
* Explain different use cases for the Sequential and Functional APIs

---

## Table of Contents

---

## Computer Vision

In this introduction to Convolutional Neural Networks (CNNs), the focus is on the transformative power of computer vision and the technical necessity of convolutions when dealing with high-resolution image data.

### Importance and Impact of Computer Vision

* **Rapid Advancement:** Deep learning has propelled computer vision into real-world utility, enabling self-driving cars, advanced face recognition, and relevant content curation in consumer apps.
* **Cross-Fertilization:** Architectural innovations in computer vision often inspire breakthroughs in other fields, such as speech recognition.

### Key Computer Vision Problems

* **Image Classification:** Determining whether an object (e.g., a cat) is present in an image.
* **Object Detection:** Not only identifying objects but also determining their specific positions and drawing bounding boxes around them.
* **Neural Style Transfer:** Repainting a content image in the artistic style of a reference image (e.g., turning a landscape photo into a "Picasso" style painting).

<img src='images/cv.png' width=750px>

### The Challenge of Input Scale

* **Small Images:** A $64 \times 64$ RGB image has 12,288 features ($64 \times 64 \times 3$), which is manageable for standard fully connected networks.
* **Large Images:** A modest $1000 \times 1000$ (1-megapixel) image results in 3,000,000 input features.
* **Parameter Explosion:** * In a fully connected layer with just 1,000 hidden units, a 1-megapixel input would require a weight matrix with 3 billion parameters.
* **Overfitting:** With billions of parameters, models are highly prone to overfitting without massive amounts of data.
* **Resource Constraints:** The memory and computational power required to train such a network are generally infeasible for standard hardware.

<img src='images/input_scale.png' width=750px>

### The Solution: Convolutional Operations

* To process high-resolution images efficiently without a parameter explosion, deep learning uses **Convolutional Neural Networks (CNNs)**.
* **Convolutions:** This operation is the fundamental building block of CNNs, allowing the network to learn local patterns (like edges) while drastically reducing the number of parameters compared to fully connected layers.